In [440]:
import logging
import random
import itertools
from collections import OrderedDict
from collections import namedtuple
from operator import index
import random
from matplotlib import pyplot as plt
from random import randrange

In [441]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]


def TotalLength(genome):
    ConvertToList = [j for i in genome for j in i]
    if len(ConvertToList)==0:
        return 0, 0
    return len(set(ConvertToList)), len(ConvertToList)


def GenerationPupulation(PROBLEM_SPACE, PopulationSize, N):
    population = []

    for i in range(PopulationSize):
        # select the sets size of the population
        listSize = randrange(N*2)
        selectedTupels = set()
        for j in range(listSize):
            selectedTupels.add(PROBLEM_SPACE[randrange(len(PROBLEM_SPACE) - 1)])
        if len(selectedTupels) != 0:
            population.append(Individual(selectedTupels, TotalLength(selectedTupels)))
    return population


def tournament(population, tournament_size=2):
    return sorted(random.choices(population, k=tournament_size), key=lambda i: (-i.fitness[0], i.fitness[1]))[0]


def cross_over(g1, g2):
    p1cut = random.randint(0, len(g1.genome))
    p2cut = random.randint(0, len(g2.genome))
    g1 = [j for i in g1 for j in i][:-2]
    g2 = [j for i in g2 for j in i][:-2]
    son = g1[:p1cut] + g2[p2cut:]
    return set(son)

In [442]:
logging.getLogger().setLevel(logging.INFO)

In [443]:
SEED = 42
N = [5, 10, 20, 100, 500, 1000]
for SetCount in N:
    PROBLEM_SPACE = list(set(tuple(i) for i in problem(SetCount, seed=SEED)))
    # N -> POPULATION_SIZE, OFFSPRING_SIZE, NUM_GENERATIONS
    # 5 -> 80, 100, 100
    # 10 -> 80, 100, 200
    # 20 -> 80, 100, 1000
    # 100 -> 80, 100, 1000
    # 500 -> 80, 100, 1000
    # 1000 -> 80, 100, 1000
    NUM_GENERATIONS = 1000
    OFFSPRING_SIZE = 100
    POPULATION_SIZE =  80
    Individual = namedtuple("Individual", ["genome", "fitness"])
    pupulation = GenerationPupulation(PROBLEM_SPACE, 80, SetCount)
    mutationRate = 30
    for g in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            if g % mutationRate == 0:
                    p1 = tournament(pupulation)
                    p1.genome.add(PROBLEM_SPACE[randrange(len(PROBLEM_SPACE) - 1)])
                    offspring.append(Individual(p1.genome, TotalLength(p1.genome)))

            else:
                p1 = tournament(pupulation)
                p2 = tournament(pupulation)
                selectedTupels = cross_over(p1, p2)
                offspring.append(Individual(selectedTupels, TotalLength(selectedTupels)))
        pupulation += offspring
        pupulation = sorted(pupulation, key=lambda i: (-i.fitness[0], i.fitness[1]))[:POPULATION_SIZE]

    individual = pupulation[0]
    w = individual.fitness[1]
    logging.info(
        f"GE solution for N={SetCount}: w={w} (bloat={(w - SetCount) / SetCount * 100:.0f}%)"
    )

INFO:root:GE solution for N=5: w=5 (bloat=0%)
INFO:root:GE solution for N=10: w=10 (bloat=0%)
INFO:root:GE solution for N=20: w=24 (bloat=20%)
INFO:root:GE solution for N=100: w=185 (bloat=85%)
INFO:root:GE solution for N=500: w=1590 (bloat=218%)
INFO:root:GE solution for N=1000: w=3559 (bloat=256%)
